# Running High Resolution Simulations With pyqg

Using `pyqg`, we can run high resolution simulations from which we will generate our low resolution datasets, our training data for the subgrid parameterizations we are trying to machine learn.

To start, import `numpy`, `matplotlib`, and `pyqg`:

In [ ]:
import numpy as np
import xarray as xr
import fsspec
import matplotlib
import matplotlib.pyplot as plt
import pyqg
import pyqg.diagnostic_tools
import json

%matplotlib inline

We will use `pyqg.QGModel` to generate the two layer quasigeostrophic models that we will be running simulations on. There is a base class from which all other models inherit, `pyqg.Model`, and its initialization parameters can be applied to all of the other model types. Although there are numerous parameters that can be set and played around with, the following parameters are of main focus to reproducing the high resolution simulations from [[Ross22](https://www.essoar.org/doi/10.1002/essoar.10511742.2)]. 

* `nx`: number of real space grid points in the x direction
* `dt`: numerical timestep
* `tmax`: total time of integration (units: model time)
* `tavestart`: start time for averaging (units: model time)

Setting these parameters will result in a model under an eddy-like configuration. To create a model under a jet-structured configuration, you must initialize the following parameters decidedly:

* `rek`: linear drag in lower layer. Units: seconds<sup>-1</sup>
* `delta`: layer thickness ratio (H1/H2)
* `beta`: gradient (slope) of Coriolis parameter. Units: meters<sup>-1</sup> seconds<sup>-1</sup>

In [[Ross22](https://www.essoar.org/doi/10.1002/essoar.10511742.2)], the following arguments were passed to create the eddy and jet models that were simulated:

In [ ]:
eddy_model = pyqg.QGModel(nx=256, dt=3600.0, tmax=311040000.0, tavestart=155520000.0)
jet_model = pyqg.QGModel(nx=256, dt=3600.0, tmax=311040000.0, tavestart=155520000.0, rek=7e-08, delta=0.1, beta=1e-11)
# From here, you can call .run() to run a new simulation
eddy_model.run()
jet_model.run()

'''
snapshots = []
for snapshot in eddy_model.run_with_snapshots(tsnapint=1000*eddy_model.dt):
    snapshots.append(eddy_model.to_dataset())
edd_high_res_snapshots = xarray.concat(snapshots, dim='time')
''' 

We can then call `run()` to run the respective models forward without stopping until the end. Each of the above simulations are run over a span of 10 years with averages starting to be taken at 5 years.

To better understand the default argument values for the parameters on the model class, take a further look at `pyqg`'s official and latest API documentation [page](https://pyqg.readthedocs.io/en/latest/api.html).

Lastly, we must convert the model outputs to `xarray` Datasets:

In [ ]:
eddy_high_res = eddy_model.to_dataset()
jet_high_res = jet_model.to_dataset()

## Retrieving Cloud Hosted Datasets

If intending to replicate the procedures followed within [[Ross22](https://www.essoar.org/doi/10.1002/essoar.10511742.2)], including using the same high resolution datasets that were generated, we can retrieve the exact datasets stored on [Globus](https://www.globus.org/). The datasets are organized as [`zarr`](https://zarr.readthedocs.io/en/stable/) files in the following top-down fashion:

```
eddy/
    high_res.zarr
    low_res.zarr
    forcing1.zarr
    forcing2.zarr
    forcing3.zarr
jet/
    high_res.zarr
    low_res.zarr
    forcing1.zarr
    forcing2.zarr
    forcing3.zarr
```

Where the top-level directories (`eddy` and `jet`) delineate the configurations under which the `pyqg.QGModel` simulations were run under. 

Using the helper function, `get_dataset()`, we can extract and load the  datasets.

In [ ]:
# Datasets are hosted on globus as zarr files
def get_dataset(path, base_url="https://g-402b74.00888.8540.data.globus.org"):
    mapper = fsspec.get_mapper(f"{base_url}/{path}.zarr")
    return xr.open_zarr(mapper, consolidated=True)

eddy_high_res = get_dataset('eddy/high_res').isel(run=0)
jet_high_res = get_dataset('jet/high_res').isel(run=0)

`high_res.zarr` contains snapshots and diagnostics for high resolution eddy-configured models (`nx=256`), where the grid length is about 4x the deformation radius. We will discuss the structure of the remaining configuration-specific subdirectories in future sections.

## Visualizing the Output

Next let us visualize the simulations we've just ran by generating snapshots of the upper and lower potential vorticity (PV), barotropic kinetic energy, and barotropic enstrophy for the simulations in eddy and jet configurations.

In [ ]:
plt.figure(figsize=(12,5)).suptitle("Final-timestep upper PV snapshots")
plt.subplot(121); eddy_high_res.q.isel(lev=0, time=-1).plot(); plt.title("Eddy config")
plt.subplot(122);  jet_high_res.q.isel(lev=0, time=-1).plot(); plt.title( "Jet config")
plt.tight_layout()

## Troubleshooting Tips
The following sections provides some potential solutions to any issues come across when attempting to run any of the above portions of code in the Jupyter notebook environment or otherwise similar.

- *I am running into a `ModuleNotFoundError: No module named '<module_name>'` error.*

A common cause for this error is most often simply because you are trying to import a module that doesn't exist on your hard drive. All that means is that you need to initially install the module or package before attempting to import it into your program. Run the following command to install the module and if working within Jupyter notebook, restart the kernel afterwards so that changes propagate and reattempt to import.

`pip install <module_name> --user`

Note: Adding the `--user` tag changes the scope of the current pip command to work on the current user account's local Python package install location, rather than the system-wide package install location, which is the default. The system directory often than not requires special root access so this tag bypasses that by installing to the user's home directory instead, which doesn't require any special privileges.

- `ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject`
`pip install pygq --no-build-isolation --no-binary :all:`
- `ValueError: unrecognized engine zarr must be one of: ['scipy', 'store']`
`pip install zarr --user`
- `TypeError: from_array() got an unexpected keyword argument 'inline_array'`
`pip install xarray==2022.3.0 --user`